In [1]:
import requests
import json
import math
import time
import os
import uuid
import platform
import logging
import subprocess
from subprocess import Popen, PIPE
import pandas as pd
from dateutil.relativedelta import relativedelta
from math import ceil
import datetime
from datetime import timedelta
from dateutil import tz
import dateutil.parser as dp
import pandas.io.gbq as gbq
from google.cloud import bigquery
from google.cloud import storage

In [2]:
app_path = os.getcwd()
gspath = 'gs://sarasmaster'
os.chdir(os.getcwd())
filesep = '\\' if platform.system() == 'Windows' else '/'
gssep = '/'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "creds" + filesep + "sarasmaster-524142bf5547.json"
gcspath = 'C:\\Users\\kabhi\\AppData\\Local\\Google\\Cloud SDK\\google-cloud-sdk\\bin'
os.environ["PATH"] += os.pathsep + gcspath
batfile = app_path + filesep + 'movetogcs.bat' if platform.system() == 'Windows' else app_path + filesep + 'movetogcs.sh'
delimitertype = 'CSV'
loadtype = 'WRITE_APPEND'
skipheader = 1

In [3]:
def week_of_month(dt):
    first_day = dt.replace(day=1)
    dom = dt.day
    adjusted_dom = dom + first_day.weekday()
    return int(ceil(adjusted_dom/7.0))

In [4]:
def loadlocalfiletogooglestorage(batfile, source_file_name, dest_file_name):
    pass_arg=[]
    pass_arg.append(batfile)
    pass_arg.append(source_file_name)
    pass_arg.append(dest_file_name)
    p = Popen(pass_arg, stdout=PIPE, stderr=PIPE)
    output, errors = p.communicate()
    p.wait() # wait for process to terminate
    print(output)
    print(errors)

In [5]:
def loadfiletobigquery(file_name, dataset_id, table_name, delimitertype, loadtype, skipheader):
    client = bigquery.Client()
    table_ref = client.dataset(dataset_id).table(table_name)
    job_config = bigquery.LoadJobConfig()
    if skipheader is not None:
        job_config.skip_leading_rows = skipheader
    job_config.source_format = delimitertype
    #if delimitertype == bigquery.SourceFormat.CSV:
        #job_config.autodetect = True
    job_config.write_disposition = loadtype

    load_job = client.load_table_from_uri(
        file_name,
        table_ref,
        job_config=job_config)  # API request

    assert load_job.job_type == 'load'

    load_job.result()  # Waits for table load to complete.

    assert load_job.state == 'DONE'

In [22]:
def addcolumnames(dfdata):
    dfdata.columns = ['business_channel','annual','month_jan','month_jan_weekly','month_feb','month_feb_weekly','month_mar','month_mar_weekly','month_apr','month_apr_weekly','month_may','month_may_weekly','month_jun','month_jun_weekly','month_jul','month_jul_weekly','month_aug','month_aug_weekly','month_sep','month_sep_weekly','month_oct','month_oct_weekly','month_nov','month_nov_weekly','month_dec','month_dec_weekly']
    return dfdata

In [35]:
def readfiles(filename):
    df = pd.read_excel(filename, sheet_name=6,header=None)
    dfdata = df[2:]
    dfdata = dfdata[:-1]
    dfdata = dfdata.reset_index(drop=True)
    dfdata = addcolumnames(dfdata)
    dfdata['budget_name'] = 'budget_v1_2018'
    start_date = datetime.datetime.now()
    localfilename = app_path + filesep + 'kopari' + filesep + 'kopari_budgets_'
    gsfilepath = gspath + gssep + 'kopari' 
    localfilename = localfilename + start_date.strftime('%Y%m%d') + '.csv'
    dfdata.to_csv(localfilename, index=False)
    loadlocalfiletogooglestorage(batfile, localfilename, gsfilepath)

In [36]:
rootdir = app_path + filesep + 'kopari' + filesep + 'budgets'
dfdata = pd.DataFrame()
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        filepath = subdir + os.sep + file
        if filepath.endswith(".xls") or filepath.endswith(".xlsx"):
            print(filepath)
            readfiles(filepath)

C:\Users\kabhi\Desktop\smd\kopari\budgets\Week Ending April 14 Retail Sales Comparisons (1).xlsx
b'\r\n(datamaster) C:\\Users\\kabhi\\Desktop\\smd>gsutil cp C:\\Users\\kabhi\\Desktop\\smd\\kopari\\kopari_budgets_20180505.csv gs://sarasmaster/kopari \r\n'
b'Copying file://C:\\Users\\kabhi\\Desktop\\smd\\kopari\\kopari_budgets_20180505.csv [Content-Type=application/vnd.ms-excel]...\r\n/ [0 files][    0.0 B/  1.8 KiB]                                                \r/ [1 files][  1.8 KiB/  1.8 KiB]                                                \r\r\nOperation completed over 1 objects/1.8 KiB.                                      \r\n'


In [19]:
df = pd.read_excel('C:\\Users\kabhi\\Desktop\\smd\kopari\\budgets\\Week Ending April 14 Retail Sales Comparisons (1).xlsx', sheet_name=6,header=None)

In [8]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Annual,Jan,Jan Weekly,Feb,Feb Weekly,Mar,Mar Weekly,Apr,Apr Weekly,...,Aug,Aug Weekly,Sept,Sept Weekly,Oct,Oct Weekly,Nov,Nov Weekly,Dec,Dec Weekly
2,Sephora US/CA,4881000,202000,48095.2,217000,51666.7,279000,66428.6,413000,98333.3,...,411000,97857.1,341000,81190.5,339000,80714.3,580000,138095,517000,123095
3,Ulta,7940000,0,0,0,0,436000,103810,436000,103810,...,844000,200952,488000,116190,488000,116190,671000,159762,561000,133571
4,Nordstrom,1174000,23000,5476.19,23000,5476.19,92000,21904.8,115000,27381,...,148000,35238.1,69000,16428.6,69000,16428.6,115000,27381,114000,27142.9


In [20]:
dfdata = df[2:]
dfdata = dfdata[:-1]
dfdata = dfdata.reset_index(drop=True)
dfheaders = df[:2]


In [21]:
dfdata.head()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,Sephora US/CA,4881000,202000,48095.2,217000,51666.7,279000,66428.6,413000,98333.3,...,411000,97857.1,341000,81190.5,339000,80714.3,580000,138095,517000,123095
1,Ulta,7940000,0,0,0,0,436000,103810,436000,103810,...,844000,200952,488000,116190,488000,116190,671000,159762,561000,133571
2,Nordstrom,1174000,23000,5476.19,23000,5476.19,92000,21904.8,115000,27381,...,148000,35238.1,69000,16428.6,69000,16428.6,115000,27381,114000,27142.9
3,Other,2790000,90000,21428.6,227000,54047.6,228000,54285.7,108000,25714.3,...,448000,106667,248000,59047.6,189000,45000,69000,16428.6,188000,44761.9
4,International,800000,0,0,0,0,45000,10714.3,76000,18095.2,...,59000,14047.6,35000,8333.33,76000,18095.2,59000,14047.6,35000,8333.33


In [22]:
end_date = dfheaders.iloc[1][9]
start_date = dfheaders.iloc[1][9] + datetime.timedelta(days=-6)
print(start_date)
print(end_date)

2017-01-15 00:00:00
2017-01-21 00:00:00


In [33]:
dfdata.head()

,new_flag,rank_wk,rank_ytd,class_name,sku_number,item_description,st,retail,net_sales_avg_we,net_sales_we,...,wos_m2,wos_m3,wos_m4,load_id,start_date,end_date,year,qtr,week,currency_code
0,NaN,1,1,160,1891746,KOP COCONUT MELT,A,38,10626.2,6521.27,...,0,0,0,f1ea4039-ca28-40ea-93f6-eeb75763d70b,2017-01-15,2017-01-21,2017,1,4,USD
1,NaN,1,1,160,1891746,KOP COCONUT MELT,A,38,7703.13,4881.62,...,0,0,0,f1ea4039-ca28-40ea-93f6-eeb75763d70b,2017-01-15,2017-01-21,2017,1,4,USD
2,NaN,1,1,160,1891746,KOP COCONUT MELT,A,38,2923.06,1639.65,...,0,0,0,f1ea4039-ca28-40ea-93f6-eeb75763d70b,2017-01-15,2017-01-21,2017,1,4,USD
3,NaN,2,2,160,1891720,KOP COCONUT SHEER OIL,A,44,2054.99,1857.77,...,NaN,NaN,NaN,f1ea4039-ca28-40ea-93f6-eeb75763d70b,2017-01-15,2017-01-21,2017,1,4,USD
4,NaN,2,2,160,1891720,KOP COCONUT SHEER OIL,A,44,1492.95,1203.11,...,NaN,NaN,NaN,f1ea4039-ca28-40ea-93f6-eeb75763d70b,2017-01-15,2017-01-21,2017,1,4,USD


In [177]:
loadlocalfiletogooglestorage(batfile, localfilename, gsfilepath)

b'\r\n(datamaster) C:\\Users\\kabhi\\Desktop\\datamaster>gsutil cp C:\\Users\\kabhi\\Desktop\\datamaster\\kopari\\nordstrom\\nordstrom_sales_20180107_20180421.csv gs://sarasmaster/kopari/nordstrom \r\n'
b'Copying file://C:\\Users\\kabhi\\Desktop\\datamaster\\kopari\\nordstrom\\nordstrom_sales_20180107_20180421.csv [Content-Type=application/vnd.ms-excel]...\r\n/ [0 files][    0.0 B/152.8 KiB]                                                \r/ [1 files][152.8 KiB/152.8 KiB]                                                \r\r\nOperation completed over 1 objects/152.8 KiB.                                    \r\n'


In [178]:
gsfilename = gsfilepath + gssep + 'nordstrom_sales_' + start_date.strftime('%Y%m%d') + '_' + end_date.strftime('%Y%m%d') + '.csv'
loadfiletobigquery(gsfilename, 'kopari', 'nordstrom_sales', delimitertype, loadtype, skipheader)

BadRequest: 400 Error while reading data, error message: CSV table encountered too many errors, giving up. Rows: 1; errors: 1. Please look into the error stream for more details.

In [99]:
def getlastupdateddate(dataset_name, table_name):
    client = bigquery.Client()
    query = "select max(to_date) max_updated_dt,max(to_date) min_updated_dt from " + dataset_name + "." + table_name + ";"
    job_config = bigquery.QueryJobConfig()
    query_job = client.query(query, job_config=job_config)

    query_job.result()  # Wait for job to complete

    # Print the results.
    destination_table_ref = query_job.destination
    table = client.get_table(destination_table_ref)
    table_data = None
    for row in client.list_rows(table):
        table_data = row
    return table_data

In [106]:
def checkexceptions(dataset_name, table_name, column_name):
    lastshopdate = getlastupdateddate('kopari', 'locad_tracking').max_updated_dt
    noexceptions = True if dfdata[(dfdata[column_name] > lastshopdate)].shape[0] > 0 else False
    return noexceptions